In [54]:
%matplotlib inline
from audiolazy import *
from librosa import load
from librosa.core.pitch import piptrack
import matplotlib.pyplot as plt
import math

# Bless https://ptolemy.berkeley.edu/eecs20/sidebars/hertz/index.html
def freq_to_rad(freq, sr):
    return (freq / sr) * (2 * pi)

def pulse_train(freq, sr, samples):
    period = 1 / freq # period in seconds
    dur = period * sr
    pulse_period = impulse(dur=dur)

    i = 0
    out = Stream([])
    while i < samples:
        if samples - i < dur:
            out.append(pulse_period.peek(samples - i))
            i = samples
        else:
            out.append(pulse_period.peek(dur))
            i += dur
            
    return out

def excitation(rad, samples):
    out = []
    
    if rad == 0.0:
        return white_noise(dur=samples).take(samples)
    
    N = int(pi // rad)
    
    d_i = 0
    for i in range(samples):
        denominator = sin_table[d_i]
        
        n_i = (((2 * N) + 1) * d_i) % DEFAULT_TABLE_SIZE
        numerator = sin_table[n_i]
        
        new_d_i = d_i + (((rad / 2) * DEFAULT_TABLE_SIZE) / (2 * pi))
        d_i = int(new_d_i) % DEFAULT_TABLE_SIZE
        
        if denominator == 0:
            out.append(1)
        else:
            out.append(numerator / denominator)
            
    return out

def get_fundamental(freqs, mags, t):
    if not t < freqs.shape[1]:
        return 0
    return freqs[mags[:, t].argmax(), t]

In [77]:
nuttiness = WavStream("nuttiness.wav")
y, sr = load("nuttiness.wav", sr=nuttiness.rate)
freqs, mags = piptrack(y=y, sr=sr, fmin=50, fmax=300, hop_length=200)

BLK_SIZE = 400

final = []
coeffs = []
resids = []
resids_rms = []

while True:
    try:
        blk = nuttiness.take(BLK_SIZE)
    except Exception as e:
        print("Done")
        break
    analysis_filt = lpc.covar(blk, 5)
    coeffs.append(analysis_filt)
    residual = analysis_filt(blk)
    resids.append(residual.peek(BLK_SIZE))
    resids_rms.append(list(envelope(Stream(residual.peek(BLK_SIZE)))))
    synth_filt = 1 / analysis_filt
    amplified_blk = synth_filt(Stream(residual))
    final += amplified_blk.peek(BLK_SIZE)

with AudioIO(True) as player: # True means "wait for all sounds to stop"
  player.play(final, rate=sr, channels=2)

Done


In [78]:
i = 0
final = []
while i < len(coeffs):
    if i == len(resids) - 1:
        break
        
    filt = coeffs[i]
    synth_filt = 1 / filt
#     amplified_blk = list(synth_filt(resids[i])) + list(synth_filt(resids[i+1]))
    excitation_pulse = excitation(freq_to_rad(get_fundamental(freqs, mags, i), sr), BLK_SIZE)
    excitation_signal = [pulse * rms for pulse, rms in zip(excitation_pulse, resids_rms[i])]
    amplified_blk = list(synth_filt(excitation_signal))
    final += amplified_blk
    
    i += 1
    
with AudioIO(True) as player: # True means "wait for all sounds to stop"
  player.play(final, rate=sr, channels=2)